In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tourism-data/test/test.xlsx
/kaggle/input/tourism-data/test/result1.csv
/kaggle/input/tourism-data/train/2018-2019.xlsx
/kaggle/input/tourism-data/train/2020-2021.xlsx


In [2]:
from matplotlib import pyplot as plt
import seaborn as sns
import gensim
import pyLDAvis
import warnings
import jieba
import re
%matplotlib inline
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [3]:
data1 = pd.read_excel('/kaggle/input/tourism-data/train/2018-2019.xlsx', sheet_name=4)
data2 = pd.read_excel('/kaggle/input/tourism-data/train/2020-2021.xlsx', sheet_name=4)

In [4]:
data1.head()

,文章ID,公众号标题,发布时间,正文
0,1001,2018，对自己好一点,2018-01-02 17:28,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...
1,1002,春节机票预订有窍门,2018-01-02 17:28,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...
2,1003,冬日旅游知多D,2018-01-03 17:32,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...
3,1004,2018冬季暖心之旅,2018-01-03 17:32,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明,2018-01-05 16:57,


In [5]:
data2.head()

,文章ID,公众号标题,发布时间,正文
0,1622,元旦晚8点，茂名海鲜亮相央视！一定要看！,2020-01-01 19:22,新年伊始，你的打开方式是怎样的呢？是准备去享受一顿丰盛的大餐？还是将“世界那么大，我想去看看...
1,1623,建好省级全域旅游示范区，打造沿海经济带新增长极 茂名市电白区旅游协会举行第二届第二次全体会议,2020-01-05 02:10,会员报到\n\n会场\n\n 为了更好地贯彻落实区委区政府和区文广旅体局建设全...
2,1624,【茂名交投旅游】@茂名人，信宜旅游景点上新啦~,2020-01-06 17:27,好消息！好消息！\n\n\n信宜莲花湖庄园\n\n开业啦~~~\n庄园简介\n\n\n\n\...
3,1625,【茂名交投旅游】2020，打卡鼎龙湾新玩法！,2020-01-06 17:27,2020.首波惊喜狂潮\n鼎龙湾·德萨斯牛仔小镇全面升级\n1月11日\n牛仔主题嘉年华街区...
4,1626,茂名 | 串珠成链，建滨海旅游目的地,2020-01-08 21:35,地处粤港澳大湾区、北部湾城市群、海南自贸区三大国家战略交汇处的茂名，紧紧抓住沿海经济带与大湾...


In [6]:
columns = ['ID', 'Title', 'Date', 'Content']
data1.columns = columns
data2.columns = columns

In [7]:
data = pd.concat([data1, data2], ignore_index=True)
data['Content'] = data['Title'] + '\n' + data['Content']
data['Date'] = pd.to_datetime(data['Date'])
data.drop(columns=['Title'], inplace=True)
data.head()

,ID,Date,Content
0,1001,2018-01-02 17:28:00,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...
1,1002,2018-01-02 17:28:00,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...
2,1003,2018-01-03 17:32:00,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...
3,1004,2018-01-03 17:32:00,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...
4,1005,2018-01-05 16:57:00,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...


In [8]:
data.dropna(inplace=True)

In [9]:
def preprocessing(data):
    lines = data.split()
    lines = list(filter(lambda x: x is not '', lines))
    unuse_lis = []
    rule_1 = r'\W'
    compiled_rule_1 = re.compile(rule_1)
    for line in lines:
        no_en_and_da = compiled_rule_1.findall(line)
        no_en_and_da_str = ''.join(no_en_and_da)
        reslis = re.findall(r'^\S', ''.join(re.findall(r'[^\，]', ''.join(re.findall(r'[^\。]', no_en_and_da_str)))))
        unuse_lis.append(reslis)
    syms = []
    for i in unuse_lis:
        for j in i:
            syms.append(j)
    syms = list(set(syms))
    
    def replace_syms(line):
        for sym in syms:
            line = line.replace(sym, '')
        return line
    
    def replace_lem(line):
        a = re.sub(r'\s', '', line)
        b = re.sub(r'\W{2,}', '', a)
        c = re.sub(r' ', '', b)
        return c
    
    lines = list(map(replace_syms, lines))
    lines = list(map(replace_lem, lines))
    lines = list(filter(lambda x: x not in '1234567890', lines))
    res = []
    for line in lines:
        for word in jieba.cut(line):
            if word == '，':
                continue
            res.append(word)
    return ' '.join(res)

In [10]:
data['Cut'] = data['Content'].apply(preprocessing)
data.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.978 seconds.
Prefix dict has been built successfully.


,ID,Date,Content,Cut
0,1001,2018-01-02 17:28:00,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...,2018 对 自己 好 一点 2017 的 旅程 已经 结束 2018 的 未来 拉开 了 ...
1,1002,2018-01-02 17:28:00,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...,春节 机票 预订 有 窍门 距离 春节 还有 一个多月 的 时间 在线 旅游 网站 的 春节...
2,1003,2018-01-03 17:32:00,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...,冬日 旅游 知多 D 960 万平方公里 的 祖国 大地 四季 都 有 独特 美景 冬天 的...
3,1004,2018-01-03 17:32:00,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...,2018 冬季 暖心 之 旅 长 按 二维码 关注 我们 中心 联系人 林小姐 137096...
4,1005,2018-01-05 16:57:00,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...,关于 粤 K27618 号 大客车 排气管 喷火 ” 事件 的 情况 说明


In [11]:
data.shape

(6285, 4)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(data['Cut'])
feature_names = vectorizer.get_feature_names()
matrix = tf_idf.toarray()
feature_names_df = pd.DataFrame(matrix, columns=feature_names)
feature_names_df

,00,000,0000,00002,00009,0002,0003,0005,0006,0007,...,龟场,龟尾园,龟约,龟缩,龟背竹,龟苓,龟苓膏,龟苗,龟裂,龟鳖
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
